In [ ]:
from collections import namedtuple
import re

In [ ]:
s = "x::a"
pos = rindex(s, "::")

In [ ]:
pos

In [ ]:
def rindex(s, sub):
    return len(s) - s[::-1].index(sub) - len(sub)

#: extended signature RE: with explicit module name separated by ::
class IrregularExpression(object):
    FakeMatch = namedtuple('FakeMatch', 'groups')
    py_sig = re.compile(
        r'''^     (\w.*?) \s*               # symbol
                  (?:
                      \((.*)\)              # optional: arguments
                      (?:\s* -> \s* (.*))?  # return annotation
                  )? $
              ''', re.VERBOSE)

    def match(self, full):
        m = self.py_sig.match(full)
        if not m:
            return None
        s, arg, retann = m.groups()
        # Extract module name using a greedy match.
        explicit_modname = None
        if "::" in s:
            pos = rindex(s, "::") + 2
            explicit_modname = s[:pos]
            s = s[pos:]
            print((explicit_modname, s))
        pieces = []
        piece = ''
        num_open = 0
        i = 0
        for c in s:
            if c.isspace() or c == '(':
                break
            if num_open == 0 and c == '.':
                pieces.append(piece)
                piece = ''
            else:
                if c == '[':
                    num_open += 1
                elif c == ']':
                    num_open -= 1
                piece += c
            i += 1
        if not piece:
            return None
        pieces.append(piece)
        path = '.'.join(pieces[:-1])
        if path:
            path += "."
        else:
            path = None
        base = pieces[-1]
        groups = lambda: (explicit_modname, path, base, arg, retann)
#         if "[" not in s_orig:
#             try:
#                 assert old_m is not None, (s_orig, groups())
#                 old_groups = old_m.groups()
#                 assert old_groups == groups(), s_orig
#                 print(groups())
#                 print(old_groups)
#                 print("---")
#             except:
#                 import traceback
#                 traceback.print_stack()
#                 raise
        # print(("match", s_orig, groups()))
        return self.FakeMatch(groups)

py_ext_sig_re = IrregularExpression()

In [ ]:
#: extended signature RE: with explicit module name separated by ::
old = re.compile(
    r'''^ ([\w.]+::)?            # explicit module name
          ([\w.]+\.)?            # module and/or class name(s)
          (\w+)  \s*             # thing name
          (?: \((.*)\)           # optional: arguments
           (?:\s* -> \s* (.*))?  #           return annotation
          )? $                   # and nothing more
          ''', re.VERBOSE)

In [ ]:
names = [
    "a.b.Name",
    "a.b.Name[Hello]",
    "x::a",
    "x::a.b.Name[Stuff.Bye].Yawrs[x[1.2]](self, x, y) -> None",
    "FindClosestPair(lane: drake::maliput::api::Lane, ego_pose: pydrake.systems.rendering.PoseVector, traffic_poses: pydrake.systems.rendering.PoseBundle, scan_distance: float, path_or_branches: pydrake.automotive.ScanStrategy) -> Dict[pydrake.automotive.AheadOrBehind, pydrake.automotive.ClosestPose]",
]

def g(x):
    if x: return x.groups()

for name in names:
    print(g(py_ext_sig_re.match(name)))
    print(g(old.match(name.replace('[', '_').replace(']', '_'))))
    print("---")
    

In [ ]:
import sys

In [ ]:
sys.path